In [40]:
# For Reproducable Code
from numpy.random import seed
seed(42)
import tensorflow
tensorflow.random.set_seed(42)

# other standard imports. "pip install -r requirements.txt" to install dependencies and "pip freeze > requirements.txt" to update them
from collections import Counter
import pickle

import PIL
import scipy
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import image_dataset_from_directory
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from dbn import SupervisedDBNClassification

In [3]:
# Useful Functions
def pklSave(contentToBeSaved, fullPath):
    with open(fullPath, 'wb') as f:
        pickle.dump(contentToBeSaved, f)

def pklLoad(fullPath, convertToNumpyArray=False):
    with open(fullPath, 'rb') as f:
        content = pickle.load(f)
    if convertToNumpyArray:
        content = np.array(content)
    return content

def train_val_test_split(x, y, tr, va, te, **kwargs):
    x_train, x_test_val, y_train, y_test_val = train_test_split(x, y, test_size=1-tr)
    x_val, x_test, y_val, y_test = train_test_split(x_test_val, y_test_val, test_size=te/(te+va), **kwargs)
    return (x_train, x_val, x_test, y_train, y_val, y_test)

# Fitting the model on Rescaled Images

In [28]:
x = pklLoad('../x.pkl', convertToNumpyArray=True)
y = pklLoad('../y.pkl', convertToNumpyArray=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [21]:
len(x_train), len(x_test), len(y_train), len(y_test)

(1865, 467, 1865, 467)

In [35]:
x_train.shape

(1865, 16384)

In [61]:
classifier = SupervisedDBNClassification(hidden_layers_structure=[100, 100],
                                         learning_rate_rbm=0.05,
                                         learning_rate=0.1,
                                         n_epochs_rbm=3,
                                         n_iter_backprop=5,
                                         batch_size=32,
                                         activation_function='sigmoid',
                                         dropout_p=0.2)

In [62]:
# classifier.fit(x_train, y_train)

[START] Pre-training step:
>> Epoch 1 finished 	RBM Reconstruction error 0.000029
>> Epoch 2 finished 	RBM Reconstruction error 0.000004
>> Epoch 3 finished 	RBM Reconstruction error 0.000004
>> Epoch 1 finished 	RBM Reconstruction error 0.001275
>> Epoch 2 finished 	RBM Reconstruction error 0.000321
>> Epoch 3 finished 	RBM Reconstruction error 0.000143
[END] Pre-training step
[START] Fine tuning step:
>> Epoch 1 finished 	ANN training loss 11.630318
>> Epoch 2 finished 	ANN training loss 11.334755
>> Epoch 3 finished 	ANN training loss 11.585020
>> Epoch 4 finished 	ANN training loss 11.406383
>> Epoch 5 finished 	ANN training loss 11.232209
[END] Fine tuning step


SupervisedDBNClassification()

Output of the fitting: <br>
<img src='notebook_media/dbn1output.png' width=400 />

In [67]:
classifier.save('dbnModel1.pkl')

In [68]:
classifier = SupervisedDBNClassification.load('dbnModel1.pkl')

In [69]:
y_pred = classifier.predict(x_train)

In [70]:
print(f'Done.\nTraining Accuracy: {accuracy_score(y_train, y_pred)}')

Done.
Training Accuracy: 0.17158176943699732


In [71]:
y_pred = classifier.predict(x_test)

In [72]:
print(f'Done.\nTesting Accuracy: {accuracy_score(y_test, y_pred)}')

Done.
Testing Accuracy: 0.15417558886509636


# Fitting the model on Features Extracted From Rescaled Images